In [4]:
import os
import sys
import numpy as np
import PIL.Image as pil 
import matplotlib.pyplot as plt
from multiprocessing import Pool

PROJ_DIR = os.path.dirname(os.path.abspath('.'))
if PROJ_DIR not in sys.path:
    sys.path.append(PROJ_DIR)

In [2]:
def compute_errors(gt, pred, mask=None):
    """Computation of error metrics between predicted and ground truth depths
    """
    if mask is None:
        mask = np.ones_like(gt) == 1
        
    gt = gt[mask]
    pred = pred[mask]
        
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

In [ ]:
model_output = dict()
model_output["SGBM"] = '/scratch/shantanu.singh/bev_models/output/occ_ant/{}/{}.png'
model_output["Anynet SGBM"] = '/scratch/shantanu.singh/bev_models/output/mb_-_bsim/dump/0/{}/0/left_bevpred/{}.png'
model_output["Anynet GT"] = '/scratch/shantanu.singh/bev_models/output/mb_dgt_bsim/dump/0/{}/0/left_bevpred/{}.png'

In [ ]:
gt_rgb = '/scratch/shantanu.singh/HabitatGibson/data/{}/0/left_rgb/{}.jpg'
gt_depth = '/scratch/shantanu.singh/HabitatGibson/data/{}/0/left_depth/{}.png'

In [5]:
val_split_file = '../splits/gibson/gibson_val_depth.txt'
with open(val_split_file, 'r') as f:
    val_fps = f.read().splitlines()
    
val_fps[:5]

['Darden 1', 'Darden 2', 'Darden 3', 'Darden 4', 'Darden 5']

In [ ]:
model_scores = {}

for filepath in eval_filepaths:
    folder, fileidx = filepath.split()

    gt_path = gt_depth.format(folder, fileidx)
    gt = np.array(pil.open(gt_path), dtype=np.float32)/6553.5 + 1e-6


    for modelname in model_output:
        
        pred_path = model_output[modelname].format(folder, fileidx)
        pred = pil.open(pred_path)
        
        if model_name in ['SGBM']:
            pred = np.array(pred, dtype=np.float32)/1000.0 + 1e-6

        median_scale = np.median(gt) / np.median(pred)
        scaled_pred = pred * median_scale

        mask = None

    scores.append(compute_errors(gt, scaled_pred, mask))